# Craigslist data extraction

## 1. Importing libraries

In [530]:
import pandas as pd
import numpy as np
import requests
import bs4
import re
import datetime
import string
import json

This is the basis url, but we are going to use other eventually:

In [2]:
url = "https://berlin.craigslist.org/d/housing/search/hhh"
response = requests.get(url)

In [3]:
html_bytes = response.content

In [5]:
type(soup)

bs4.BeautifulSoup

In [525]:
soup.body;

In [31]:
a= soup.find_all('p', class_="result-info") # Here is the relevant info

This is the first result of our data:

In [106]:
a[0]

<p class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">post zu favoriten</span>
</span>
<time class="result-date" datetime="2020-02-15 09:10" title="Sa 15 Feb 09:10:16">Feb 15</time>
<a class="result-title hdrlnk" data-id="7060721562" href="https://berlin.craigslist.org/roo/d/furnished-unique-ethnic-style-apt-for/7060721562.html">Furnished, Unique, Ethnic Style Apt- For short or long period!!!!!€400</a>
<span class="result-meta">
<span class="result-price">€400</span>
<span class="result-hood"> (berlin)</span>
<span class="result-tags">
</span>
<span class="banish icon icon-trash" role="button">
<span class="screen-reader-text">dieses posting verbergen</span>
</span>
<span aria-hidden="true" class="unbanish icon icon-trash red" role="button"></span>
<a class="restore-link" href="#">
<span class="restore-narrow-text">wiederherstellen</span>
<span class="restore-wide-text">dieses posting wiederherstellen</span>
</a>
</span>
</p>

However, in this we don't have information about the area and number of bedrooms, so we try a different one.

In [107]:
a[1]

<p class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">post zu favoriten</span>
</span>
<time class="result-date" datetime="2020-02-15 06:12" title="Sa 15 Feb 06:12:01">Feb 15</time>
<a class="result-title hdrlnk" data-id="7057016829" href="https://berlin.craigslist.org/vac/d/top-floor-apartment-with-terrace-march/7057016829.html">TOP FLOOR APARTMENT with terrace - MARCH 2020</a>
<span class="result-meta">
<span class="result-price">€1000</span>
<span class="housing">
                    1br -
                    70m<sup>2</sup> -
                </span>
<span class="result-hood"> (Berlin, Mitte)</span>
<span class="result-tags">
<span class="pictag">Bild</span>
</span>
<span class="banish icon icon-trash" role="button">
<span class="screen-reader-text">dieses posting verbergen</span>
</span>
<span aria-hidden="true" class="unbanish icon icon-trash red" role="button"></span>
<a class="restore-link" href="#">
<span class="restore-narrow-text

This give us the url, which is not important but just for exploring:

In [88]:
a[1].a["href"]

'https://berlin.craigslist.org/vac/d/top-floor-apartment-with-terrace-march/7057016829.html'

Obtaining the price:

In [89]:
a[1].find("span", class_="result-price").text

'€1000'

In [114]:
a[6]

<p class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">post zu favoriten</span>
</span>
<time class="result-date" datetime="2020-02-14 17:34" title="Fr 14 Feb 17:34:29">Feb 14</time>
<a class="result-title hdrlnk" data-id="7075147658" href="https://berlin.craigslist.org/rew/d/snapchat-sessions-findom-worship/7075147658.html">Snapchat sessions/ findom worship</a>
<span class="result-meta">
<span class="result-price">€0</span>
<span class="result-tags">
</span>
<span class="banish icon icon-trash" role="button">
<span class="screen-reader-text">dieses posting verbergen</span>
</span>
<span aria-hidden="true" class="unbanish icon icon-trash red" role="button"></span>
<a class="restore-link" href="#">
<span class="restore-narrow-text">wiederherstellen</span>
<span class="restore-wide-text">dieses posting wiederherstellen</span>
</a>
</span>
</p>

The Neighborhood:

In [117]:
a[1].find("span", class_= "result-hood").text

' (Berlin, Mitte)'

In [131]:
b= a[1].find("span", class_= "result-hood").text


In [135]:
b.replace("(", "").replace(")", "").lstrip()

'Berlin, Mitte'

The description:

In [91]:
a[1].find("a").text

'TOP FLOOR APARTMENT with terrace - MARCH 2020'

The number of bedrooms and area:

In [93]:
a[1].find("span", class_= "housing").text

'\n                    1br -\n                    70m2 -\n                '

The date in which the apartment was posted:

In [109]:
a[1].find("time", class_= "result-date").text

'Feb 15'

In [111]:
a[1].time["datetime"]

'2020-02-15 06:12'

And, obviously, the unique identifier for each single apartment:

In [94]:
a[1].a["data-id"]# obtain data-ids

'7057016829'

# Creating Functions for extract data

In [119]:
def get_name(list_item):
    id_apartment = list_item.a["data-id"]
    return id_apartment


In [120]:
get_name(a[0])

'7060721562'

In [264]:
def get_description(list_item):
    try:
        id_description = list_item.find("a").text
        return id_description.capitalize()
    except:
        return ""


In [273]:
get_description(a[9])

'Gorgeous, furnished 1 br flat in neukölln near maybachufer'

In [184]:
def get_date(list_item):
    return list_item.time["datetime"].split()[0]

In [276]:
get_date(a[15])

'2020-02-14'

In [314]:
def get_price(list_item):
    try:
        return list_item.find("span", class_="result-price").text
    except:
        return "€0"

In [317]:
get_price(a[6])

'€0'

In [130]:
get_price(a[2])

'€1000'

In [284]:
def get_location(list_item):
    try:
        location = list_item.find("span", class_= "result-hood").text
        location = location.replace("(", "").replace(")", "").lstrip()
        return string.capwords(location)
    except:
        return ""

In [292]:
get_location(a[100])

'Prenzlauer Berg, Berlin'

In [301]:
def get_size(list_item):
    try:
        size_bedrooms = list_item.find("span", class_= "housing").text
        size_bedrooms= size_bedrooms.lstrip().split()
        size_bedrooms = [i for i in size_bedrooms if i != "-"]       
        #return size_bedrooms
        if len(size_bedrooms) == 2:
            return size_bedrooms[1].replace("m2", "")
        else:
            if size_bedrooms[0].find("m2") != -1:
                return size_bedrooms[0].replace("m2", "")
            else:
                return 0
    except:
        return 0

### this should be done with try!!! if the length is 

In [308]:
get_size(a[3])

0

In [309]:
def get_bedrooms(list_item):
    try:
        size_bedrooms = list_item.find("span", class_= "housing").text
        size_bedrooms= size_bedrooms.lstrip().split()
        size_bedrooms = [i for i in size_bedrooms if i != "-"]       
        #return size_bedrooms
        if len(size_bedrooms) == 2:
            return size_bedrooms[0].replace("br", "")
        else:
            if size_bedrooms[0].find("br") != -1:
                return size_bedrooms[0].replace("br", "")
            else:
                return 0
    except:
        return 0

In [310]:
get_bedrooms(a[3])

'2'

Now, we will construct a function joining all the single fields for our dataset:

In [318]:
def get_record(list_item):
    return {"date_id": get_date(list_item),
            "apartment_id": get_name(list_item),
            "location": get_location(list_item),
            "price": get_price(list_item),
            "size": get_size(list_item),
            "bedrooms": get_bedrooms(list_item),
            "description": get_description(list_item)}

In [319]:
get_record(a[1])

{'date_id': '2020-02-15',
 'apartment_id': '7057016829',
 'location': 'Berlin, Mitte',
 'price': '€1000',
 'size': '70',
 'bedrooms': '1',
 'description': 'Top floor apartment with terrace - march 2020'}

In [352]:
df_berlin = pd.DataFrame([get_record(list_item) for list_item in a])

In [353]:
df_berlin.head()

,date_id,apartment_id,location,price,size,bedrooms,description
0,2020-02-15,7060721562,Berlin,€400,0,0,"Furnished, unique, ethnic style apt- for short..."
1,2020-02-15,7057016829,"Berlin, Mitte",€1000,70,1,Top floor apartment with terrace - march 2020
2,2020-02-15,7057017433,"Berlin, Mitte",€1000,70,1,Top floor apartment with terrace - march 2020
3,2020-02-14,7075441143,Berlin,€850,0,2,43mq new and cosy flat in rummelsburg / lichte...
4,2020-02-14,7075310096,Berlin Kreuzberg Alte Brauerei,€4500,0,4,"Roof top paradise kreuzberg (4 rooms, open kit..."


In [356]:
df_berlin.T.to_json("apartments.json")

In [358]:
df_berlin.T

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
date_id,2020-02-15,2020-02-15,2020-02-15,2020-02-14,2020-02-14,2020-02-14,2020-02-14,2020-02-14,2020-02-14,2020-02-14,...,2020-02-13,2020-02-13,2020-02-13,2020-02-13,2020-02-13,2020-02-13,2020-02-13,2020-02-13,2020-02-13,2020-02-13
apartment_id,7060721562,7057016829,7057017433,7075441143,7075310096,7066187786,7075147658,7063873498,7058753216,7069362147,...,7057262211,7056874679,7074461856,7062807281,7074461459,7074457166,7059222244,7071325256,7065294791,7057701982
location,Berlin,"Berlin, Mitte","Berlin, Mitte",Berlin,Berlin Kreuzberg Alte Brauerei,Berlin,,Neltestraße,Berlin Friedrichshain,"Bürknerstraße, Neukölln",...,"Friedrichshain, Berlin",Berlin Mitte,Berlin,Friedrichstraße,"Kreuzberg, Berlin",Schönhauser Allee,,,Mitte,Germany Berlin Friedrichshain
price,€400,€1000,€1000,€850,€4500,€500,€0,€589,€399,€400,...,€500,€1000,€400,€500,€725,€499,€390,€500,€500,€390
size,0,70,70,0,0,0,0,0,0,0,...,0,700,0,52,37,53,52,45,52,0
bedrooms,0,1,1,2,4,1,0,0,0,0,...,0,2,0,0,1,1,0,1,1,0
description,"Furnished, unique, ethnic style apt- for short...",Top floor apartment with terrace - march 2020,Top floor apartment with terrace - march 2020,43mq new and cosy flat in rummelsburg / lichte...,"Roof top paradise kreuzberg (4 rooms, open kit...",Cosy & lavishly equipped studio apartment for ...,Snapchat sessions/ findom worship,Room for rent in apartment with four bedrooms ...,Updated price awesome 1 br apt with large patio,"Gorgeous, furnished 1 br flat in neukölln near...",...,Newly renovated charming apartment for rent,Downtown best area in mitte - linienstr (mitte...,Fully furnished home with amenities inclusive,+++apartment near checkpoint charlie in mitte+++,I am moving abroad and looking for somebody to...,--brand new 1 bedroom apartment-- at schönhaus...,----->cosy & confortable 1 bedroom situated in...,Rooftop studio in neukölln near volkspark hase...,Lovely 1 bedroom apartment near checkpoint cha...,"Cable/ internet ready, secured parking, 1 bedr..."


In [360]:
def get_data_frame(web):
    response = requests.get(web)
    html_bytes = response.content
    suppe = bs4.BeautifulSoup(html_bytes, "html.parser")
    items = suppe.find_all('p', class_="result-info")
    return pd.DataFrame([get_record(list_item) for list_item in items])
    

In [361]:
def get_url(city):
    return "https://"+city+".craigslist.org/search/hhh"

In [405]:
cities = ["amsterdam", "berlin", "bremen", "dusseldorf", "hamburg", "hannover", "leipzig", "munich"]

In [ ]:
df[~df.C.str.contains("XYZ")]

In [509]:
def get_dataf(city):
    web = get_url(city)
    data_f = get_data_frame(web)
    city_s = str(city).capitalize()
    data_f["city"] = data_f["location"].apply(lambda x: city_s)
    return data_f    

In [510]:
final_data = pd.concat([get_dataf(city) for city in cities])

In [511]:
final_data = final_data.reset_index()

In [512]:
final_data.rename(columns = {"index": "no"}, inplace = True)

In [513]:
final_data.drop(columns = "no", inplace = True)

In [514]:
final_data[final_data.location.str.contains("Berlin")]

,date_id,apartment_id,location,price,size,bedrooms,description,city
120,2020-02-15,7075753678,Charlottenburg- Berlin,€0,45,0,"Berlinale, two-room suite, own entrance + bike",Berlin
122,2020-02-15,7063040204,Charlottenburg- Berlin,€1000,0,0,"Two room suite with own entrance, furnished",Berlin
123,2020-02-15,7075739124,"Kollwitzstraße, Berlin, Germany",€300,52,1,1x1 sun-flooded luxury studio in prime locatio...,Berlin
124,2020-02-15,7063254959,"Reichenberger Str., Friedrichshain-kreuzberg, ...",€350,0,1,%%%fabulous! remodeled one bedroom! make it yo...,Berlin
135,2020-02-15,7060721562,Berlin,€400,0,0,"Furnished, unique, ethnic style apt- for short...",Berlin
...,...,...,...,...,...,...,...,...
236,2020-02-13,7057772664,"Mitte, Berlin",€399,0,0,Updated tranqil 1br apartment for rent,Berlin
237,2020-02-13,7056391549,"Neukölln, Berlin",€300,0,0,Freshly remodeled studio apartment,Berlin
238,2020-02-13,7056391237,"Mitte, Berlin",€300,0,0,Cosy & equipped studio apartment for rent,Berlin
239,2020-02-13,7056866890,Berlin Mitte,€1000,65,2,Downtown best area in mitte - linienstr (mitte...,Berlin


In [516]:
final = final_data[~(final_data.price.str.contains("CZK") | final_data.price.str.contains("CHF"))]

In [518]:
final.shape

(931, 8)

In [523]:
final.drop_duplicates(subset='apartment_id', keep="first").tail()

,date_id,apartment_id,location,price,size,bedrooms,description,city
934,2020-02-12,7073902931,,€450,52,1,Apartment for rent one bedroom nuremberg,Munich
935,2020-02-15,7075698285,,€790,0,2,Familienfreundliche 2-zimmer-wohnung,Munich
936,2020-02-13,7074422466,,€475,45,1,Apartment in a central location,Munich
950,2020-02-14,7075251254,,€500,0,1,1 bed immediate move in available,Munich
951,2020-02-14,7075251204,,€500,0,1,1 bed immediate move in available,Munich


In [471]:
cities2 = ["amsterdam", "bremen", "dusseldorf", "hamburg", "hannover", "leipzig", "munich"]

In [472]:
rest_data = pd.concat([get_dataf(city) for city in cities2])

In [473]:
rest_data = rest_data.reset_index()

In [474]:
rest_data.rename(columns = {"index": "no"}, inplace = True)

In [475]:
rest_data.drop(columns = "no", inplace = True)

In [476]:
rest_data.T.to_json("rest_data.json")

In [478]:
final_data.shape

(960, 7)

In [480]:
final_data.tail()

,date_id,apartment_id,location,price,size,bedrooms,description
955,2020-02-14,7075243449,Munich,CZK11400,51,1,Fabulous one bedroom apartment for rent in prague
956,2020-02-14,7060786325,Munich,CZK15000,70,1,Beautiful 2 rooms apartment + large balcony
957,2020-02-13,7074786279,Munich,CZK13600,63,2,* charming two bedrooms apartment for rent in ...
958,2020-02-13,7074789229,Munich,CZK13600,63,2,* charming two bedrooms apartment for rent in ...
959,2020-02-13,7074787772,Munich,CZK13600,63,2,* charming two bedrooms apartment for rent in ...


In [486]:
final = final_data[~(final_data.price.str.contains("CZK") | final_data.price.str.contains("CHF"))]

In [487]:
final.head()

,date_id,apartment_id,location,price,size,bedrooms,description
0,2020-02-15,7065109521,Slotervaart Near Lelylaan Station,€2100,145,2,"Huge house 2 double bedrooms patio, garage and..."
1,2020-02-15,7060797134,Amsterdam,€80,0,2,Full two-bedroom apartment in the heart of ams...
2,2020-02-15,7060798681,Amsterdam,€80,0,2,Newly renovated top floor apartment in a quiet...
3,2020-02-15,7062656915,Amsterdam,€1200,0,1,Cozy apartment in a quiet street near the sing...
4,2020-02-15,7062658353,Amsterdam,€40,0,1,Century apartment in the famous jordaan area i...


In [497]:
final.location.value_counts()

Bremen                    111
Leipzig                   111
Hannover                  110
Dusseldorf                103
Hamburg                    94
                         ... 
Palamedesstraat             1
Robert-koch-straße          1
Sankt Johann                1
Am Schafbrinke 71           1
Sacramento, California      1
Name: location, Length: 193, dtype: int64